# 3. Hands-on experiment:  Computing a geometric embedding for the Sparsest Cut Problem via Semidefinite Programming (20 pts)


The Uniform Sparsest Cut problem (USC) aims to find a bipartition $(S, \bar{S})$ of the nodes of a graph $G = (V, E)$, $\vert V \vert = p$, which minimizes the quantity 
\begin{equation*}
\label{sceq}
       \frac{E(S, \bar{S})}{\lvert S \rvert \; \lvert \bar{S} \rvert}, 
\end{equation*}
where $E(S, \bar{S})$ is the number of edges connecting $S$ and $\bar{S}$, and $\lvert S \rvert $ is the number of nodes in $S$.

Computing such a bipartition is NP-hard and intense research has gone into designing efficient approximation algorithms for this problem. In the seminal work of Arora, Rao and Vazirani (2009), an $\mathcal{O}(\sqrt{\log p})$ approximation algorithm is proposed for solving USC, which relies on finding a well-spread $\ell_2^2$ geometric representation of $G$ where each node $i\in V$ is mapped to a vector $\mathbf{v}_i$ in $\mathbb{R}^p$. In this experimental section we focus on solving the SDP that computes this geometric embedding. 

### SDP formulation
The canonical formulation of the SDP is
\begin{align}
\mathbf{X}^\star \in \arg \min_{ \mathbf{X} } \bigg\{ \langle \boldsymbol{C}, \mathbf{X} \rangle : &~p \; \mathrm{Tr}(\mathbf{X}) -  \mathrm{Tr}(\mathbf{1}_{p\times p}\mathbf{X}) = \frac{p^2}{2}, \\\\ 
& \; \mathbf{X}_{i,j} + \mathbf{X}_{j,k} - \mathbf{X}_{i, k} - \mathbf{X}_{j,j} \leq 0, \; \forall\ i \neq j \neq k \neq i\in V, \\\\
&~\underbrace{\mathrm{Tr}(\mathbf{X}) \leq p, ~\mathbf{X} \in \mathbb{R}^{p\times p} ,~\mathbf{X}\succeq 0}_{\mathcal{X}} \bigg\},
\end{align}
where $\boldsymbol{C}$ represents the Laplacian of graph $G$ and $\mathbf{X}_{i,j} = \langle \mathbf{v}_{i}, \mathbf{v}_j\rangle$ gives the geometric embedding of the nodes. We can rewrite the above SDP as


\begin{equation}
    \label{eq:reform-sdp}
    \min_{\mathbf{X} \in \mathcal{X}} f(\mathbf{X}) + g(A(\mathbf{X})) ~~~~~~~~\text{ subject to } ~~~~B_{i,j,k}(\mathbf{X}) \in \mathcal{K}, \;\forall\ i \neq j \neq k \neq i\in V,
\end{equation}
where $f(\mathbf{X}) = \langle \boldsymbol{C}, \mathbf{X} \rangle$ and  $g(\cdot) = \delta_{\left\{\frac{p^2}{2}\right\}}(\cdot)$ is the indicator function of singleton $\left\{\frac{p^2}{2}\right\}$.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse.linalg import eigsh, svds, eigs
from math import sqrt
import scipy.io as sio
import random
import numpy.matlib
from helpers import *
import time
from scipy.sparse import isspmatrix
import itertools

# fix the seed
random.seed( 3 )

In [ ]:
# Load data
#dataset = 'data/102n-insecta-ant-colony4-day10.mat'
#dataset = 'data/55n-insecta-ant-colony1-day37.mat'
dataset = 'data/25mammalia-primate-association-13.mat'

Problem = sio.loadmat(dataset)
C = np.double(Problem['LAPL']) # euclidean distance matrix
p = np.int(Problem['d']) # number of data points
opt_val = Problem['opt_val'] # optimum value 
X_true = Problem['X_true']
nchoosek_inds = list(itertools.combinations(range(p), 3))


#### <span style="font-variant:small-caps;">3.(c) *(10 points)*</span>

We will now observe the behavior of HCGM on three graphs from the Network Repository dataset \cite{nr}: 
* **G1 (25 nodes)**: *mammalia-primate-association-13*, 
* **G2 (55 nodes)**: *55n-insecta-ant-colony1-day37*
* **G3 (102 nodes)**: *insecta-ant-colony4-day10*

$\triangleright$ Fill in the missing code and run the algorithm.

$\triangleright$ Add resulting plots to your report.

$\triangleright$ What do you notice about the running times of the algorithm for the three problem instances?

$\triangleright$ What are the potential bottlenecks to applying this method to large graphs?


In [ ]:
def grad_F(X, beta):
    # Add the equality constraint
    [grad_val, displacement] = get_equality_constr_grad(X, beta)
    feas_eq = abs(displacement)
   
    # Add the triangle constraints. This needs to be scaled by d to match
    # the order of the equality constr
    n = len(nchoosek_inds)
    feas_ineq = 0
    for l in range(n):
        index_triplet = nchoosek_inds[l]
        [update_vector, row_idxs, col_idxs, part_feas_normsq] = get_triangle_constr_grad(X, beta, index_triplet)
        
        grad_val[row_idxs, col_idxs] = grad_val[row_idxs, col_idxs] + update_vector
        grad_val[col_idxs, row_idxs] = grad_val[col_idxs, row_idxs] + update_vector
        
        feas_ineq = feas_ineq + part_feas_normsq
    
    # Total gradient
    grad_val = grad_val + C
    feas_ineq = sqrt(feas_ineq)
    
    return (grad_val, feas_eq, feas_ineq)



def get_equality_constr_grad(X, beta):
    # X is a p x p matrix
    displacement = p * np.trace(X) - np.sum(X) - p**2/2
    grad_val = (p/beta) * (p * np.eye(p) - np.ones((p, p))) * displacement
    
    return (grad_val, displacement)



def get_triangle_constr_grad(X, beta, index_triplet):
    # X needs to be in a matrix shape, not vectorized
    #print(index_triplet)
    i = index_triplet[0]
    j = index_triplet[1]
    k = index_triplet[2]
    t1 = max(X[i, j] + X[j, k] - X[i, k] - X[j, j], 0.)
    t2 = max(X[i, k] + X[i, j] - X[j, k] - X[i, i], 0.)
    t3 = max(X[i, k] + X[j, k] - X[i, j] - X[k, k], 0.)

    grad_update = (1. / beta) * np.array([t1 + t2 - t3, # for Xij 
                                -t1 + t2 + t3, # for Xik
                                t1 - t2 + t3, # for Xjk
                                -t2/2., # for Xii - ./2 because we update these twice when we symmetrize
                                -t1/2., # for Xjj - ./2 because we update these twice when we symmetrize
                                -t3/2.]) # for Xkk - ./2 because we update these twice when we symmetrize

    update_row_idxs = [i, i, j, i, j, k]
    update_col_idxs = [j, k, k, i, j, k]
    part_feas_normsq = 2. * (t1**2 + t2**2 + t3**2)
    
    return (grad_update, update_row_idxs, update_col_idxs, part_feas_normsq)

In [ ]:
# Plotting function
def plot_func(cur_iter, feasibility1,feasibility2, objective, X, X_true):
    plt.figure(figsize=(12, 8))
    plt.subplot(221)
    plt.loglog(cur_iter, feasibility1)#, 'go--', linewidth=2, markersize=12))
    plt.xlabel('iteration',fontsize=15)
    plt.ylabel('$\|A(X) - b\|$',fontsize=15)
    plt.grid(True)

    plt.subplot(222)
    plt.loglog(cur_iter, feasibility2)
    plt.xlabel('iteration',fontsize=15)
    plt.ylabel('dist$(B(X), (-\infty, 0])$',fontsize=15)
    plt.grid(True)
    plt.show()

    #plt.subplot(223)
    obj_res = np.reshape(np.abs(objective - opt_val)/opt_val, (len(objective),))
    plt.figure(figsize=(12, 8))
    plt.loglog((cur_iter), (obj_res))
    plt.xlabel('iteration',fontsize=15)
    plt.ylabel('$(f(X) - f^*)/f^*$',fontsize=15)
    plt.title('Relative objective residual',fontsize=15)
    plt.grid(True)
    plt.show()
    
    plt.figure(figsize=(12, 8))
    plt.subplot(221)
    plt.imshow(X)
    plt.title('HCGM solution',fontsize=15)
    plt.colorbar()
    
    plt.subplot(222)
    plt.imshow(X_true)
    plt.title('True solution',fontsize=15)
    plt.colorbar()
    plt.show()

In [ ]:
def HCGM(kappa=p, maxit=np.int(1e3), beta0=1e2):
    # Initialize
    X = np.zeros((p,p))
    
    feasibility1 = [] # norm(A1(X)-b1)/norm(b1)
    feasibility2 = [] # dist(X, \mathcal{K})
    objective    = [] # f(x)
    cur_iter    = [] 
    t    = [] 
    
    #u = np.zeros((N,1))
    iter_track = np.unique(np.ceil(np.power(2, np.linspace(0,20,50))))
    
    start = time.time()
    
    for iteration in range(1, maxit+1):
        
        # Update Step Size
        gamma = ??
        
        # Update beta
        beta_ = ??
        
        # Write down the vk to use in the lmo (eigenvalue routine)
        (vk, feas_eq, feas_ineq) = grad_F(X, beta_)
        # the above needs to be symmetric
        
        
        # Linear minimization oracle
        q, u = eigsh(??, k=1, tol=1e-16, which='SA')
        if q >= 0:
            X_bar = ??
        else:
            u = sqrt(kappa)*u
            X_bar = np.outer(u,u)
        
        # Update X
        X = (1.0 - gamma) * X + gamma * (X_bar)
                
        if any(iteration == iter_track) or iteration==maxit:
            feasibility1.append(feas_eq)
            feasibility2.append(feas_ineq)
            objective.append(np.sum(C.flatten()*X.flatten()))
            cur_iter.append(iteration)
            t.append(time.time()-start)
            print('{:03d} | {:.4e}| {:.4e}| {:.4e}|'.format(iteration, feasibility1[-1], feasibility2[-1],objective[-1]))
            
    return X, feasibility1, feasibility2, objective, cur_iter, t

In [ ]:
print("\nDataset = {}. \n".format(dataset))
print("\n f_opt = {}. \n".format(opt_val))

X_HCGM, f1_HCGM, f2_HCGM, obj_HCGM, iter_HCGM, time_HCGM = HCGM(kappa=p, maxit=np.int(1e3), beta0=5e1)

print("\nHCGM finished. Running time: {} seconds.".format(time_HCGM[-1]))

In [ ]:
plot_func(iter_HCGM, f1_HCGM, f2_HCGM, obj_HCGM, X_HCGM, X_true)